In [1]:
#imports
import carla #Carlas library
import random #Allows for randomisation
import cv2 #working with images from camera/sensors
import numpy as np 

In [2]:
client = carla.Client('localhost', 2000) #connects to the open carla client using port 2000

In [4]:
client.load_world('Town04')

RuntimeError: time-out of 5000ms while waiting for the simulator, make sure the simulator is ready and connected to localhost:2000

In [ ]:
client.load_world('Town05')

In [ ]:
#Gets the world aka the environment/world and all the possible spawn points
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()

In [ ]:
#Gets the blueprint of Mini car
vehicle_bp = world.get_blueprint_library().filter('*mini*')

In [ ]:
#After getting the spawn points from the world I can using the
#random library to get a random spawn point
#It then spawns a vehicle at that spawn point
start_point = random.choice(spawn_points)
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

In [ ]:
#Set the spectators location at where the vehicle was spawned
spectator = world.get_spectator()
start_point.location.z = start_point.location.z+1
spectator.set_transform(start_point)

In [ ]:
#Sets vehicle to autopilot
vehicle.set_autopilot(True)

In [ ]:
#setting RGB camera using CARLA Video 
#Link - https://www.youtube.com/watch?v=om8klsBj4rc

#Camera offset mounted to the car
CAMERA_POS_Z = 1.6 #1.6mm from z axis (up from ground)
CAMERA_POS_X = 0.9 #0.9mm from x axis (forward)

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640')
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))

#Creates a camera in the simulator
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))
    
image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
#Opens a live feed from camera
camera.listen(lambda image: camera_callback(image,camera_data))


In [ ]:
#Saves a snapshot from the camera
img = camera_data['image']
cv2.imshow('RGB Camera',img)
cv2.waitKey(0)

In [ ]:
#Cleaning up

camera.stop()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    actor.destroy()